In [1]:
import numpy as np
from typing import Tuple, Dict
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))
import pandas as pd
import tensorflow as tf
from tensorflow.math import logical_not
from builderfuncs import EarlyStopAndSave
from builderfuncs import build_gru, build_bigru, build_bilstm, build_lstm, save_whole_model, restore_model
from parameterdicts import GRUParameters
from ktfuncs import *
import keras_tuner as kt
import os
import datetime


df = pd.read_csv('data/scaled_U2_data.csv', index_col=0)
df.drop("UNNAMED: 0", axis=1, inplace=True)

devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

print("Num GPUs Available: ", len(devices))

Num GPUs Available:  1


In [2]:
def ttv_split(ds: tf.data.Dataset, ds_size: int, 
              train_split: float = 0.8, 
              val_split: float = 0.1, 
              test_split: float = 0.1) -> Tuple[tf.data.Dataset, 
                                                tf.data.Dataset, 
                                                tf.data.Dataset]:

    assert (train_split + test_split + val_split) == 1
   
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds


def split_window(features: tf.Tensor) -> Tuple[Dict[tf.Tensor, tf.Tensor],
                                               tf.Tensor]:
    inputs = features[:12, 1:]
    state_labels = features[12, 75:]
    targ_labels = tf.expand_dims(features[11, 0], axis=0)
    labels = tf.concat([targ_labels, state_labels], axis=0)
    inputs.set_shape([12, 247])
    labels.set_shape([174])
    

    return inputs, labels

                        
def make_dataset(data: pd.DataFrame, length: int, 
                 batch_size: int = 64, multistep: bool = True) -> Tuple[tf.data.Dataset, 
                                                                        tf.data.Dataset, 
                                                                        tf.data.Dataset]:
    data = np.array(data.iloc[:, :], dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(data=data,
                                                        targets=None,
                                                        sequence_length=length,
                                                        sequence_stride=1,
                                                        shuffle=True,
                                                        seed=1,
                                                        batch_size=None)

    ds = ds.filter(lambda x: tf.reduce_all(logical_not(tf.math.is_nan(x))))
    ds = ds.map(split_window).batch(batch_size)
    ds = ds.apply(tf.data.experimental.assert_cardinality(170680//batch_size + 1))
    train_ds, val_ds, test_ds = ttv_split(ds, 170680//batch_size, train_split=0.8, val_split=0.1, test_split=0.1)
    train_ds = train_ds
    return train_ds, val_ds, test_ds

In [3]:
gru_logdir = os.path.join("logs", 'gru-logs')
gru_tensorboard_callback = tf.keras.callbacks.TensorBoard(gru_logdir, histogram_freq=1)
gru_earlystopper = EarlyStopAndSave(filepath="model_folder/gru_folder", patience=15, lim=0.005, minormax="max")

bigru_logdir = os.path.join("logs", 'bigru-logs')
bigru_tensorboard_callback = tf.keras.callbacks.TensorBoard(bigru_logdir, histogram_freq=1)
bigru_earlystopper = EarlyStopAndSave(filepath="model_folder/bigru_folder", patience=15, lim=0.005, minormax="max")

lstm_logdir = os.path.join("logs", 'lstm-logs')
lstm_tensorboard_callback = tf.keras.callbacks.TensorBoard(lstm_logdir, histogram_freq=1)
lstm_earlystopper = EarlyStopAndSave(filepath="model_folder/lstm_folder", patience=15, lim=0.005, minormax="max")

bilstm_logdir = os.path.join("logs", 'bilstm-logs')
bilstm_tensorboard_callback = tf.keras.callbacks.TensorBoard(bilstm_logdir, histogram_freq=1)
bilstm_earlystopper = EarlyStopAndSave(filepath="model_folder/bilstm_folder", patience=15, lim=0.005, minormax="max")

train_data, val_data, test_data = make_dataset(df, 23, batch_size=64)

In [4]:
gru_tuner = kt.BayesianOptimization(kt_gru,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="gru_trial_1",
                                    seed=1)
bigru_tuner = kt.BayesianOptimization(kt_bigru,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="bigru_trial_1",
                                    seed=1)
lstm_tuner = kt.BayesianOptimization(kt_lstm,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="lstm_trial_1",
                                    seed=1)
bilstm_tuner = kt.BayesianOptimization(kt_bilstm,
                                    objective=kt.Objective('val_loss', 'min'),
                                    directory='directory',
                                    project_name="bilstm_trial_1",
                                    seed=1)



INFO:tensorflow:Reloading Tuner from directory\gru_trial_1\tuner0.json
INFO:tensorflow:Reloading Tuner from directory\bigru_trial_1\tuner0.json
INFO:tensorflow:Reloading Tuner from directory\lstm_trial_1\tuner0.json
INFO:tensorflow:Reloading Tuner from directory\bilstm_trial_1\tuner0.json


In [5]:
parameters: GRUParameters = gru_tuner.get_best_hyperparameters()[0].values
parameters['look_back'] = parameters.pop('lookback')
gru = build_gru(parameters=parameters, name="gru_model", dynamic=False)

parameters: GRUParameters = bigru_tuner.get_best_hyperparameters()[0].values
parameters['look_back'] = parameters.pop('lookback')
bigru = build_bigru(parameters=parameters, name="bigru_model", dynamic=False)

In [6]:
gru.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[gru_earlystopper, gru_tensorboard_callback])
bigru.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[bigru_earlystopper, bigru_tensorboard_callback])

Epoch 1/200
2132/2132 - 45s - loss: 0.0066 - val_loss: 0.0021 - 45s/epoch - 21ms/step
Epoch 2/200
2132/2132 - 38s - loss: 0.0025 - val_loss: 0.0017 - 38s/epoch - 18ms/step
Epoch 3/200
2132/2132 - 42s - loss: 0.0022 - val_loss: 0.0016 - 42s/epoch - 20ms/step
Epoch 4/200
2132/2132 - 45s - loss: 0.0021 - val_loss: 0.0015 - 45s/epoch - 21ms/step
Epoch 5/200
2132/2132 - 44s - loss: 0.0020 - val_loss: 0.0014 - 44s/epoch - 21ms/step
Epoch 6/200
2132/2132 - 43s - loss: 0.0019 - val_loss: 0.0014 - 43s/epoch - 20ms/step
Epoch 7/200
2132/2132 - 42s - loss: 0.0018 - val_loss: 0.0013 - 42s/epoch - 20ms/step
Epoch 8/200
wait: 1
2132/2132 - 43s - loss: 0.0018 - val_loss: 0.0013 - 43s/epoch - 20ms/step
Epoch 9/200
2132/2132 - 42s - loss: 0.0017 - val_loss: 0.0013 - 42s/epoch - 20ms/step
Epoch 10/200
2132/2132 - 46s - loss: 0.0017 - val_loss: 0.0012 - 46s/epoch - 22ms/step
Epoch 11/200
wait: 1
2132/2132 - 47s - loss: 0.0016 - val_loss: 0.0013 - 47s/epoch - 22ms/step
Epoch 12/200
wait: 2
2132/2132 - 45s

In [7]:
parameters: GRUParameters = lstm_tuner.get_best_hyperparameters()[0].values
parameters['look_back'] = parameters.pop('lookback')
lstm = build_lstm(parameters=parameters, name="lstm_model", dynamic=False)

parameters: GRUParameters = bilstm_tuner.get_best_hyperparameters()[0].values
parameters['look_back'] = parameters.pop('lookback')
bilstm = build_bilstm(parameters=parameters, name="bilstm_model", dynamic=False)

In [8]:
lstm.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[lstm_earlystopper, lstm_tensorboard_callback])
bilstm.fit(train_data, epochs=200, verbose=2, validation_data=val_data, callbacks=[bilstm_earlystopper, bilstm_tensorboard_callback])

Epoch 1/200
2132/2132 - 63s - loss: 0.0055 - val_loss: 0.0019 - 63s/epoch - 30ms/step
Epoch 2/200
2132/2132 - 60s - loss: 0.0023 - val_loss: 0.0016 - 60s/epoch - 28ms/step
Epoch 3/200
2132/2132 - 62s - loss: 0.0020 - val_loss: 0.0015 - 62s/epoch - 29ms/step
Epoch 4/200
2132/2132 - 66s - loss: 0.0018 - val_loss: 0.0014 - 66s/epoch - 31ms/step
Epoch 5/200
2132/2132 - 64s - loss: 0.0017 - val_loss: 0.0013 - 64s/epoch - 30ms/step
Epoch 6/200
wait: 1
2132/2132 - 66s - loss: 0.0017 - val_loss: 0.0013 - 66s/epoch - 31ms/step
Epoch 7/200
2132/2132 - 59s - loss: 0.0016 - val_loss: 0.0012 - 59s/epoch - 28ms/step
Epoch 8/200
wait: 1
2132/2132 - 57s - loss: 0.0016 - val_loss: 0.0012 - 57s/epoch - 27ms/step
Epoch 9/200
2132/2132 - 56s - loss: 0.0015 - val_loss: 0.0012 - 56s/epoch - 26ms/step
Epoch 10/200
2132/2132 - 58s - loss: 0.0015 - val_loss: 0.0012 - 58s/epoch - 27ms/step
Epoch 11/200
wait: 1
2132/2132 - 58s - loss: 0.0015 - val_loss: 0.0012 - 58s/epoch - 27ms/step
Epoch 12/200
2132/2132 - 58s

In [9]:
from ToApps.to_apps import to_slack
to_slack("DONE\n START NEXT PROGRAM")